In [23]:
from pyomo.environ import ConcreteModel, SolverFactory, TransformationFactory
from pyomo.network import Arc, SequentialDecomposition
import pyomo.environ as env

from idaes.core import FlowsheetBlock

# Import properties and units from "WaterTAP Library"
from water_props import WaterParameterBlock
#from model_example import UnitProcess
from source_example import Source
from split_test import Split
from mixer_example import Mixer1

import financials

import watertap as wt

from pyomo.environ import ConcreteModel, SolverFactory, TerminationCondition, \
    value, Var, Constraint, Expression, Objective, TransformationFactory, units as pyunits
from pyomo.network import Arc, SequentialDecomposition
from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Mixer, Pump

from idaes.generic_models.unit_models import Separator as Splitter

from idaes.core.util.model_statistics import degrees_of_freedom
from pyomo.util.check_units import assert_units_consistent
import pyomo.util.infeasible as infeas
import idaes.core.util.scaling as iscale

#### This workbook includes the ability to: add a unit process, a source, and use, a splitter, and a mixer. The mixer can take multiple inlets with one outlet. The splitter on inlet and ONLY TWO outlets. Nano and chlorination tested only. Optimization works by unfixing variables such as the splitter fraction.

In [24]:
def add_unit_process(m = None, unit_process_name = None, unit_process_type = None, with_connection = False,
                     from_splitter = False, splitter_tream = None,
                    link_to = None, link_from = None, connection_type = "series", stream_name = None): # in design
        
#     if unit_process_type == "nanofiltration":
#         import nanofiltration_twb as up_module

#     if unit_process_type == "chlorination":
#         import chlorination_twb as up_module        
    
    up_module = wt.module_import.get_module(unit_process_name)
    
    setattr(m.fs, unit_process_name, up_module.UnitProcess(default={"property_package": m.fs.water}))
    
    m = up_module.create(m, unit_process_name)
    
    if with_connection == True:
        
        if from_splitter == True:
            
            setattr(m.fs, splitter_tream, 
                    Arc(source=getattr(m.fs, link_from).outlet1, 
                                           destination=getattr(m.fs, link_to).inlet))            
            
        if from_splitter == False:
            m = connect_blocks(m = m, up1 = link_from, up2 = link_to, 
                               connection_type = connection_type, 
                               stream_name = stream_name)
    
    return m
    

In [25]:
def connect_blocks(m = None, up1 = None, up2 = None, connection_type = None, stream_name = None): # in design
    
    #m.fs.stream1 = Arc(source=m.fs.NF01.outlet, destination=m.fs.NF02.inlet)
    
    setattr(m.fs, stream_name, 
            Arc(source=getattr(m.fs, up1).outlet, 
                                   destination=getattr(m.fs, up2).inlet))
            
    return m

In [26]:
def add_water_source(m = None, source_name = None, link_to = None, 
                     reference = None, water_type = None, case_study = None, flow = 0): # in design
    
    import importfile
    
    df = importfile.feedwater(
        input_file="data/case_study_water_sources_and_uses.csv",
        reference = reference, water_type = water_type, 
        case_study = case_study)
    
    setattr(m.fs, source_name, Source(default={"property_package": m.fs.water}))
    
    getattr(m.fs, source_name).inlet.flow_vol.fix(flow)
    getattr(m.fs, source_name).inlet.conc_mass[:, "TOC"].fix(df.loc["TOC"].Value)
    getattr(m.fs, source_name).inlet.conc_mass[:, "nitrates"].fix(df.loc["Nitrate"].Value) #TODO ChangeNitrate
    getattr(m.fs, source_name).inlet.conc_mass[:, "TDS"].fix(df.loc["TDS"].Value)
    getattr(m.fs, source_name).inlet.temperature.fix(300)
    getattr(m.fs, source_name).inlet.pressure.fix(2e5)
    
    
    # Set removal and recovery fractions -> CAN WE JUST SET OUTLETS AND THAT'S IT? OR CLEANER WITH THE SAME FORMAT?
    getattr(m.fs, source_name).water_recovery.fix(1.0)
    getattr(m.fs, source_name).removal_fraction[:, "TDS"].fix(1e-4)
    # I took these values from the WaterTAP3 nf model
    getattr(m.fs, source_name).removal_fraction[:, "TOC"].fix(1e-4)
    getattr(m.fs, source_name).removal_fraction[:, "nitrates"].fix(1e-4)
    # Also set pressure drops - for now I will set these to zero
    getattr(m.fs, source_name).deltaP_outlet.fix(1e-4)
    getattr(m.fs, source_name).deltaP_waste.fix(1e-4)
    
    if link_to is not None: # TODO - potential for multiple streams
        connect_blocks(m = m, up1 = source_name, up2 = link_to, connection_type = None, 
                               stream_name = ("%s_stream" % source_name))
        
    return m

In [27]:
def add_water_use(m = None, split_name = None, with_connection = False,
                    link_to = None, link_from = None, stream_name = None): # in design
    

    setattr(m.fs, split_name, Use(default={"property_package": m.fs.water}))
           
    # Set removal and recovery fractions -> CAN WE JUST SET OUTLETS AND THAT'S IT? OR CLEANER WITH THE SAME FORMAT?
    getattr(m.fs, split_name).water_recovery.fix(1)
    getattr(m.fs, split_name).removal_fraction[:, "TDS"].fix(0.5)
    # I took these values from the WaterTAP3 nf model
    getattr(m.fs, split_name).removal_fraction[:, "TOC"].fix(0.5)
    getattr(m.fs, split_name).removal_fraction[:, "nitrates"].fix(0.5)
    # Also set pressure drops - for now I will set these to zero
    getattr(m.fs, split_name).deltaP_outlet1.fix(1e-4)
    getattr(m.fs, split_name).deltaP_outlet2.fix(1e-4)
    
    if link_to is not None: # TODO - potential for multiple streams
        connect_blocks(m = m, up1 = link_from, up2 = link_to, connection_type = None, 
                               stream_name = stream_name)
        
    return m

In [28]:
def add_splitter(m = None, split_name = None, with_connection = False,
                    link_to = None, link_from = None, stream_name = None): # in design
    

    setattr(m.fs, split_name, Split(default={"property_package": m.fs.water}))
           
    # Set removal and recovery fractions -> CAN WE JUST SET OUTLETS AND THAT'S IT? OR CLEANER WITH THE SAME FORMAT?
    getattr(m.fs, split_name).water_recovery.unfix() #(0.5)
    getattr(m.fs, split_name).removal_fraction[:, "TDS"].fix(0.5)
    # I took these values from the WaterTAP3 nf model
    getattr(m.fs, split_name).removal_fraction[:, "TOC"].fix(0.5)
    getattr(m.fs, split_name).removal_fraction[:, "nitrates"].fix(0.5)
    # Also set pressure drops - for now I will set these to zero
    getattr(m.fs, split_name).deltaP_outlet1.fix(1e-4)
    getattr(m.fs, split_name).deltaP_outlet2.fix(1e-4)
    
    if link_to is not None: # TODO - potential for multiple streams
        connect_blocks(m = m, up1 = link_from, up2 = link_to, connection_type = None, 
                               stream_name = stream_name)
        
    return m

In [29]:
# -----------------------------------------------------------------------------
# Create a Pyomo model
m = ConcreteModel()

# Add an IDAES FlowsheetBlock and set it to steady-state
m.fs = FlowsheetBlock(default={"dynamic": False})

# Add water property package
m.fs.water = WaterParameterBlock()

In [30]:
# TODO -> link to multiple processes
m = add_water_source(m = m, source_name = "source1", link_to = None, 
                     reference = "Poseidon", water_type = "Wastewater", 
                     case_study = "Typical untreated domestic wastewater",
                     flow = 1000)

In [31]:
m = add_splitter(m = m, split_name = 'split1', with_connection = True,
                    link_to = 'split1', link_from = 'source1', stream_name = 'source_stream')

In [32]:
m = add_unit_process(m = m, unit_process_name = "CL01", unit_process_type = 'chlorination', 
                     with_connection = True, from_splitter = True, splitter_tream = "outlet1",
                    link_to = "CL01", link_from = "split1", connection_type = "series") #, stream_name = "stream1")

In [33]:
m = add_unit_process(m = m, unit_process_name = "NF01", unit_process_type = 'nanofiltration', 
                     with_connection = True, from_splitter = True, splitter_tream = "outlet2",
                    link_to = "NF01", link_from = "split1", connection_type = "series") #, stream_name = "stream2")

In [34]:
m.fs.mixer1 = Mixer1(default={"property_package": m.fs.water, "inlet_list": ["inlet1", "inlet2"]})
m.fs.arc1 = Arc(source=m.fs.NF01.outlet, destination=m.fs.mixer1.inlet1)
m.fs.arc2 = Arc(source=m.fs.CL01.outlet, destination=m.fs.mixer1.inlet2)

In [35]:
# Transform Arc to construct linking equations
TransformationFactory("network.expand_arcs").apply_to(m)

seq = SequentialDecomposition()
G = seq.create_graph(m)

In [36]:
import display
display.show_train2(G)

In [14]:
##### IF OPTIMIZING FOR PATHWAY #####
#y1 = m.fs.NF01.costing.fixed_cap_inv_unadjusted + m.fs.CL01.costing.fixed_cap_inv_unadjusted

#m.objective_function = env.Objective(expr=y1, sense=env.maximize)

# Set up a solver in Pyomo and solve
solver = SolverFactory('ipopt')
results = solver.solve(m, tee=True)

# Display the inlets and outlets of each unit
for node in G.nodes():
    print("----------------------------------------------------------------------")
    print(node)
    
    if "split" in (str(node).replace('fs.', '')): 
        getattr(m.fs, str(node).replace('fs.', '')).inlet.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet1.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet2.display()
    elif "use" in (str(node).replace('fs.', '')): 
        getattr(m.fs, str(node).replace('fs.', '')).inlet.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet1.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet2.display() 
    elif "mixer" in (str(node).replace('fs.', '')): 
        getattr(m.fs, str(node).replace('fs.', '')).inlet1.display()
        getattr(m.fs, str(node).replace('fs.', '')).inlet2.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet.display()
    else:
        getattr(m.fs, str(node).replace('fs.', '')).inlet.display()
        getattr(m.fs, str(node).replace('fs.', '')).outlet.display()
        getattr(m.fs, str(node).replace('fs.', '')).waste.display()

        
    print("Show some costing values")
    print("---------------------")
    
    if "source" in (str(node).replace('fs.', '')): 
        print("should skip:", (str(node).replace('fs.', '')))
        continue
    elif "use" in (str(node).replace('fs.', '')): 
        print("should skip:", (str(node).replace('fs.', '')))
        continue
    elif "split" in (str(node).replace('fs.', '')): 
        print("should skip:", (str(node).replace('fs.', '')))
        continue  
    elif "mixer" in (str(node).replace('fs.', '')): 
        print("should skip:", (str(node).replace('fs.', '')))
        continue
    else:
        print("should have a cost", (str(node).replace('fs.', '')))
        if getattr(m.fs, str(node).replace('fs.', '')).costing.fixed_cap_inv_unadjusted() is not None:
            print("fixed_cap_inv_unadjusted:" , 
                  getattr(m.fs, str(node).replace('fs.', '')).costing.fixed_cap_inv_unadjusted())
        else:
            getattr(m.fs, str(node).replace('fs.', '')).costing.fixed_cap_inv_unadjusted.display()
    
    print("----------------------------------------------------------------------")
    

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      236
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       43

Total number of variables............................:       85
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Tot

In [15]:
# create mixer for recycle - > two-stage process -> RO two stage.
# outlet of mixer goes into unit. outlet of unit goes to splitter, one goes to mixer, one goes to next unit.
# splitter fraction can be unfixed free variable, but random fix it to something - see RO -> 
#  
# constraint on enduse -> less than X
# first run if feasible. then cost optimization. TDS -> WHAT DO YOU GET, COST, WHAT DO YOU GET. OBJ -> MINIMIZE ERROR WITH END USE CONSRAINT
# if 0 - then can be specified, then turn into constraints. 
# 

In [16]:
m.fs.NF01.costing.fixed_cap_inv_unadjusted()

42795.87248202152

In [17]:
m.fs.CL01.costing.fixed_cap_inv_unadjusted()

2.106499577533229

In [18]:
m.fs.NF01.water_recovery.display()

water_recovery : Water recovery fraction
    Size=1, Index=fs.time
    Key : Lower : Value : Upper : Fixed : Stale : Domain
    0.0 :   0.5 :   0.8 :   1.0 :  True :  True :  Reals


In [19]:
m.fs.CL01.water_recovery.display()

water_recovery : Water recovery fraction
    Size=1, Index=fs.time
    Key : Lower : Value : Upper : Fixed : Stale : Domain
    0.0 :   0.5 :   1.0 :   1.0 :  True :  True :  Reals


In [20]:
m.fs.split1.water_recovery.display()

water_recovery : Water recovery fraction
    Size=1, Index=fs.time
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
    0.0 : 0.499 : 0.5000000000000171 : 0.501 : False : False :  Reals


In [21]:
import display
display.show_train2(G)

In [22]:
display.show_train2(G)

In [17]:
m.fs.properties = props.NaClParameterBlock()
m.fs.P1 = Pump(default={"property_package": m.fs.properties})
pump_efi = 0.75  # pump efficiency [-]

m.fs.EC = Expression(
    expr=(m.fs.P1.work_mechanical[0]))
#         / sum(m.fs.M1.mixed_state[0].flow_mass_comp[j] for j in ['H2O', 'NaCl'])
#         * m.fs.M1.mixed_state[0].dens_mass)

m.fs.P1.control_volume.properties_out[0].pressure = 50e5  # pressure out of pump 1 [Pa]

# pump 1
m.fs.P1.efficiency_pump.fix(pump_efi)
m.fs.P1.control_volume.properties_out[0].pressure.fix()  # value set in decision variables
# unfix decision variables and add bounds
m.fs.P1.control_volume.properties_out[0].pressure.unfix()  # pressure out of pump 1 [Pa]
m.fs.P1.control_volume.properties_out[0].pressure.setlb(10e5)
m.fs.P1.control_volume.properties_out[0].pressure.setub(80e5)
m.fs.P1.deltaP.setlb(0)

In [ ]:
m = connect_blocks(m = m, up1 = "source1", up2 = "NF01", connection_type = "series", stream_name = "stream1")

In [23]:
class SomeClass:
    variable_1 = "This is a class variable"
    variable_2 = 100    #this is also a class variable.

    def __init__(self, param1, param2):
        self.instance_var1 = param1
        #instance_var1 is a instance variable
        self.instance_var2 = param2   
        #instance_var2 is a instance variable

In [24]:
obj1 = SomeClass("some thing", 18) 

In [27]:
obj1.variable_1

'This is a class variable'

In [28]:
obj2 = SomeClass(28, 6)

In [29]:
obj2.variable_1

'This is a class variable'

In [31]:
obj1.instance_var2

18

In [37]:
class SomeClass:    
    def create_arr(self): # An instance method
        self.arr = []
    
    def insert_to_arr(self, value):  #An instance method
        self.arr.append(value)

In [38]:
obj3 = SomeClass()
obj3.create_arr()
obj3.insert_to_arr(5)
obj3.arr

[5]

In [31]:
class SomeClass:
    def create_arr(self): # An instance method
        self.arr = []
    
    def insert_to_arr(self, value):  #An instance method
        self.arr.append(value)
        
    @classmethod
    def class_method(cls):
        print("the class method was called")

In [32]:
SomeClass.class_method()

the class method was called


In [38]:
class SomeClass:
    def __init__(self):
        self.arr = [] 
        #All SomeClass objects will have an array arr by default
    
    def insert_to_arr(self, value):
        self.arr.append(value)

In [39]:
obj1 = SomeClass()
#obj2 = SomeClass()
#obj1.insert_to_arr(6)

In [41]:
SomeClass.insert_to_arr(obj1, "test")

In [44]:
obj1.arr[0]

'test'

In [9]:
def test1():
    
    return 2 + 2 

In [11]:
test1()

4

In [12]:
def test2():
    return test1()

In [13]:
test2()

4